In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)

In [2]:
train = pd.read_csv('training.csv')
test = pd.read_csv('testing.csv')

In [3]:
gender_map = {'F':0,'M':1}
customer_map = {'Active': 1,'Lapsed':0}
train['Gender'] = train['Gender'].map(gender_map)
train['Customer_Status'] = train['Customer_Status'].map(customer_map)
train = train.drop(['Avg_available_bal','CLIENTNUM','Total_Revolving_Bal','Relationship_age_months'], axis = 1)
train.head()

,Customer_Status,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,1,42,0,2,Graduate,Single,Less than $40K,Blue,2,1,1,3659.0,0.797,4068,61,0.694,0.000
1,1,39,1,1,Graduate,Married,$120K +,Blue,6,3,2,5764.0,1.356,1972,38,0.520,0.265
2,1,52,1,1,Graduate,Married,$80K - $120K,Blue,6,1,1,2689.0,0.378,1356,33,0.435,0.772
3,1,46,1,4,High School,Married,$80K - $120K,Blue,3,2,3,15535.0,0.461,3718,66,0.610,0.084
4,1,43,1,2,College,Married,$80K - $120K,Blue,6,1,4,1669.0,0.698,3591,59,0.788,0.000


In [4]:
from sklearn.preprocessing import OneHotEncoder
cols_to_encode = ['Education_Level','Marital_Status','Income_Category','Card_Category']
train_cat = train[cols_to_encode]
encoder = OneHotEncoder()
sparse = encoder.fit_transform(train_cat)
train_t = pd.DataFrame(sparse.toarray())
train_encoded = pd.concat([train, train_t], axis = 1)
train_encoded = train_encoded.drop(cols_to_encode, axis = 1)
train_encoded.info()
train_encoded.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Status           8101 non-null   int64  
 1   Customer_Age              8101 non-null   int64  
 2   Gender                    8101 non-null   int64  
 3   Dependent_count           8101 non-null   int64  
 4   Total_Relationship_Count  8101 non-null   int64  
 5   Months_Inactive_12_mon    8101 non-null   int64  
 6   Contacts_Count_12_mon     8101 non-null   int64  
 7   Credit_Limit              8101 non-null   float64
 8   Total_Amt_Chng_Q4_Q1      8101 non-null   float64
 9   Total_Trans_Amt           8101 non-null   int64  
 10  Total_Trans_Ct            8101 non-null   int64  
 11  Total_Ct_Chng_Q4_Q1       8101 non-null   float64
 12  Avg_Utilization_Ratio     8101 non-null   float64
 13  0                         8101 non-null   float64
 14  1       

,Customer_Status,Customer_Age,Gender,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,...,11,12,13,14,15,16,17,18,19,20
0,1,42,0,2,2,1,1,3659.0,0.797,4068,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,39,1,1,6,3,2,5764.0,1.356,1972,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,52,1,1,6,1,1,2689.0,0.378,1356,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,46,1,4,3,2,3,15535.0,0.461,3718,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,43,1,2,6,1,4,1669.0,0.698,3591,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
gender_map = {'F':0,'M':1}
customer_map = {'Active': 1,'Lapsed':0}
test['Gender'] = test['Gender'].map(gender_map)
test = test.drop(['Avg_available_bal','CLIENTNUM'], axis = 1)
test.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Relationship_age_months,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,52,0,3,Uneducated,Single,Less than $40K,Blue,41,2,1,2,1801.0,0,0.516,5047,79,0.756,0.000
1,32,1,0,Post-Graduate,Married,$60K - $80K,Blue,25,6,2,2,4469.0,2331,1.006,8314,67,0.763,0.522
2,43,1,1,Post-Graduate,Unknown,$60K - $80K,Blue,37,4,3,4,4112.0,1128,0.587,4446,75,0.667,0.274
3,40,1,4,Uneducated,Single,$80K - $120K,Blue,36,6,3,2,34516.0,572,1.050,8659,81,1.077,0.017
4,45,0,4,Uneducated,Divorced,Unknown,Blue,36,4,3,3,15875.0,1719,0.860,1592,37,1.467,0.108


In [6]:
cols_to_encode = ['Education_Level','Marital_Status','Income_Category','Card_Category']
test_cat = test[cols_to_encode]
encoder = OneHotEncoder()
sparse = encoder.fit_transform(test_cat)
test_t = pd.DataFrame(sparse.toarray())
test_encoded = pd.concat([test, test_t], axis = 1)
test_encoded = test_encoded.drop(cols_to_encode, axis = 1)
test_encoded.info()
len(train_encoded[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              2026 non-null   int64  
 1   Gender                    2026 non-null   int64  
 2   Dependent_count           2026 non-null   int64  
 3   Relationship_age_months   2026 non-null   int64  
 4   Total_Relationship_Count  2026 non-null   int64  
 5   Months_Inactive_12_mon    2026 non-null   int64  
 6   Contacts_Count_12_mon     2026 non-null   int64  
 7   Credit_Limit              2026 non-null   float64
 8   Total_Revolving_Bal       2026 non-null   int64  
 9   Total_Amt_Chng_Q4_Q1      2026 non-null   float64
 10  Total_Trans_Amt           2026 non-null   int64  
 11  Total_Trans_Ct            2026 non-null   int64  
 12  Total_Ct_Chng_Q4_Q1       2026 non-null   float64
 13  Avg_Utilization_Ratio     2026 non-null   float64
 14  0       

8101

In [7]:
X = train_encoded.drop([0,1,11],axis = 1)
y = train_encoded[1]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## DecisionTree With AdaBoosting

In [9]:
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.tree import DecisionTreeClassifier
#ada_clf = AdaBoostClassifier(
#DecisionTreeClassifier(max_depth = 1), n_estimators = 200, algorithm = 'SAMME.R', learning_rate = 0.2)
#ada_clf.fit(X_train, y_train)

In [10]:
#y_pred = ada_clf.predict(X_test)

In [11]:
#accuracy_score(y_test,y_pred)

## RandomForest

In [12]:
#from sklearn.ensemble import RandomForestClassifier
#rnd_clf = RandomForestClassifier()
#rnd_clf.fit(X_train, y_train)
#y_pred = rnd_clf.predict(X_test)

In [13]:
#from sklearn.metrics import accuracy_score
#accuracy_score(y_test,y_pred)

## Logistic Regression

In [14]:
#from sklearn.preprocessing import MaxAbsScaler
#from sklearn import utils
#transformer = MaxAbsScaler().fit(train_encoded)
#train_encoded = pd.DataFrame(transformer.transform(train_encoded))

X = train_encoded.drop([0,1],axis = 1)
y = train_encoded[1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter = 10000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [16]:
y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)

In [17]:
accuracy_score(y_test, y_pred)

0.9511352418558736

In [18]:
y_proba

array([[9.97956990e-01, 2.04301038e-03],
       [9.99656760e-01, 3.43239953e-04],
       [9.99494228e-01, 5.05772122e-04],
       [9.95225994e-01, 4.77400577e-03],
       [9.99139307e-01, 8.60693270e-04],
       [9.98203126e-01, 1.79687354e-03],
       [9.03475742e-01, 9.65242581e-02],
       [9.97137630e-01, 2.86236963e-03],
       [9.97710600e-01, 2.28940030e-03],
       [9.98326312e-01, 1.67368807e-03],
       [9.98724883e-01, 1.27511673e-03],
       [9.99118991e-01, 8.81009025e-04],
       [9.98865952e-01, 1.13404750e-03],
       [9.95716629e-01, 4.28337101e-03],
       [9.99501111e-01, 4.98888846e-04],
       [9.99132729e-01, 8.67271303e-04],
       [9.94848813e-01, 5.15118709e-03],
       [9.99036761e-01, 9.63238811e-04],
       [9.99697239e-01, 3.02760525e-04],
       [9.99803191e-01, 1.96809272e-04],
       [9.96521228e-01, 3.47877202e-03],
       [5.38201308e-01, 4.61798692e-01],
       [9.98637695e-01, 1.36230452e-03],
       [8.38179073e-01, 1.61820927e-01],
       [6.435468

In [19]:
def corr_df(x, corr_val):
    '''
    Obj: Drops features that are strongly correlated to other features.
          This lowers model complexity, and aids in generalizing the model.
    Inputs:
          df: features df (x)
          corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
    Output: df that only includes uncorrelated features
    '''

    # Creates Correlation Matrix and Instantiates
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterates through Correlation Matrix Table to find correlated columns
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = item.values
            if abs(val) >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(i)

    drops = sorted(set(drop_cols))[::-1]

    # Drops the correlated columns
    for i in drops:
        col = x.iloc[:, (i+1):(i+2)].columns.values
        x = x.drop(col, axis=1)
    return x

In [20]:
train_minus_cor = corr_df(train, 0.8)
test_minus_cor = corr_df(test,0.8)

In [21]:
train_minus_cor

,Customer_Status,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,1,42,0,2,Graduate,Single,Less than $40K,Blue,2,1,1,3659.0,0.797,4068,61,0.694,0.000
1,1,39,1,1,Graduate,Married,$120K +,Blue,6,3,2,5764.0,1.356,1972,38,0.520,0.265
2,1,52,1,1,Graduate,Married,$80K - $120K,Blue,6,1,1,2689.0,0.378,1356,33,0.435,0.772
3,1,46,1,4,High School,Married,$80K - $120K,Blue,3,2,3,15535.0,0.461,3718,66,0.610,0.084
4,1,43,1,2,College,Married,$80K - $120K,Blue,6,1,4,1669.0,0.698,3591,59,0.788,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,1,48,0,3,Uneducated,Single,Less than $40K,Blue,3,3,3,1438.3,0.544,4182,81,0.723,0.671
8097,1,62,0,1,Uneducated,Unknown,Less than $40K,Silver,6,1,3,12590.0,1.068,4182,78,0.696,0.074
8098,1,52,0,3,Graduate,Single,Less than $40K,Blue,5,2,1,2996.0,0.503,4724,66,0.737,0.717
8099,1,39,0,1,Uneducated,Married,$40K - $60K,Blue,4,1,4,3272.0,1.123,4763,86,0.593,0.710
